In [2]:
import requests
import pandas as pd
import numpy as np
import requests 
import os
from io import StringIO  # Import StringIO from the io module
import time  # Import the time module
from urllib.parse import quote  # To URL encode SMILES
import plotly.express as px
import matplotlib.colors as mcolors

Befor runing this code: 
1) creaea an empty folder (input_folder) in your desired location and writhe down the path below! 
2) Change the corresponding Q CODE from wikidata for a given Genus in the code below!

That's all

DEFINE THE FOLLOWING PARAMETERS

In [3]:
#Paths
# Input folder containing CSV files
input_folder = 'C:/Users/quirosgu/Desktop/Maytenus/Literature/' #'/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/Yggdrasil/data_out/genus/'#'C:/Users/quirosgu/Desktop/JLW/genus/'# '/mnt/c/Users/quirosgu/Desktop/JLW/'

#Swertia chirayita (Q21318003)
#Swertia (Q163970)
#Gentianaceae (Q157216)

qcode = "Q2633580"  # Replace with the actual Q-code for the genus

#lotus databases
LOTUSDB = 'C:/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv'#'/mnt/c/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv' #'/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv' #'/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv' #'C:/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv'#'/mnt/c/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv' #'/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv'
LOTUSDB_rc = 'C:/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_rc.csv'#'/mnt/c/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_rc.csv' #'C:/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_rc.csv'#'/mnt/c/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_rc.csv' #'/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/Yggdrasil/data_loc/LotusDB_inhouse_rc.csv'

# Output folder for the processed CSV files
output_folder = f'{input_folder}output_data/'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [4]:

#functions to colors palets, etc
def fetch_species_from_qcode(qcode):
    """
    Fetches all species under a given genus using the Wikidata SPARQL endpoint.

    Parameters:
    genus_qcode (str): The Wikidata Q-code for the genus.

    Returns:
    pd.DataFrame: A DataFrame containing the species and their corresponding Q-codes.
    """

    # SPARQL query to fetch species under a given genus
    query = """
    SELECT ?species ?speciesLabel WHERE {
      ?species wdt:P171* wd:%s .
      ?species wdt:P105 wd:Q7432.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """ % qcode

    # URL for the Wikidata SPARQL endpoint
    url = "https://query.wikidata.org/sparql"

    # Request headers
    headers = {
        "User-Agent": "Wikidata Species Fetcher/0.1 (https://www.wikidata.org/wiki/Wikidata:Data_access)"
    }

    # Perform the request
    response = requests.get(url, headers=headers, params={'query': query, 'format': 'json'})

    if response.status_code != 200:
        raise Exception("Failed to fetch data: HTTP status code {}".format(response.status_code))

    # Parse the JSON response
    data = response.json()

    # Extract results
    species = []
    for item in data['results']['bindings']:
        species.append({
            'wikidata_Qcode_species': item['species']['value'].split('/')[-1],
            'Species': item['speciesLabel']['value']
        })

    return pd.DataFrame(species)

# Define base colors for each Pathway
pathway_shades= {
    'Terpenoids': ('#618264', '#D0E7D2'),  # Green start and lighter green end
    'Alkaloids': ('#305F72', '#5CBCE2'),   # Blue start and lighter blue end  
    'Shikimates and Phenylpropanoids': ('#80558C', '#CBA0AE'),  # Purple start and lighter purple end
    'Polyketides': ('#EF4B4B', '#EC8F6A'),  # Red start and lighter purple end
    'Fatty acids': ('#FF6C22', '#FF9209'),  # Orange start and lighter purple end
    'Amino acids and Peptides': ('#F4E869', '#FAF2D3'),  # Yellow start and lighter purple end
    'Carbohydrates': ('#65451F','#C8AE7D')  # Brown start and lighter purple end
}

# Define custom colors for the 7 pathway categories
pathway_colors = {
    'Terpenoids': '#618264',  # Green start and lighter green end
    'Alkaloids': '#305F72',   # Blue start and lighter blue end
    'Shikimates and Phenylpropanoids': '#80558C',  # Purple start and lighter purple end
    'Polyketides': '#EF4B4B',  # Red start and lighter purple end
    'Fatty acids': '#FF6C22',  # Orange start and lighter purple end
    'Amino acids and Peptides': '#F4E869',  # Yellow start and lighter purple end
    'Carbohydrates': '#65451F' # Brown start and lighter purple end
}

def interpolate_color(color1, color2, factor: float):
    """Interpolate between two colors"""
    color1 = np.array(mcolors.to_rgb(color1))
    color2 = np.array(mcolors.to_rgb(color2))
    return mcolors.to_hex((1 - factor) * color1 + factor * color2)

def generate_shades(pathway, num_shades):
    base_color, end_color = pathway_shades.get(pathway, ('gray', 'lightgray'))
    if num_shades == 1:
        return [base_color]  # Return the base color if only one shade is requested
    shades = []
    for i in range(num_shades):
        factor = i / (num_shades - 1)
        shades.append(interpolate_color(base_color, end_color, factor))
    return shades

def split_chemical_superclass(row):
    # Check if the value is a string before splitting
    if isinstance(row['chemical_superclass'], str):
        parts = row['chemical_superclass'].split('-')
        if len(parts) == 2:
            return parts[0], parts[1]  # Pathway and Superclass are present
        else:
            return parts[0], 'Unknown'  # Only Pathway is present, or the format is not as expected
    else:
        # Return default values if the entry is NaN or not a string
        return 'Unknown', 'Unknown'

FIRST RECOVER THE SPECIES (and Q codes) BELONGING TO A PARTICULAR GENUS 

In [5]:
# Usage
species_df = fetch_species_from_qcode(qcode)
species_df.head()  # Display the first few rows

# Filename for the CSV file
file_name = "species_list.csv"

# Full path for the CSV file
full_file_path = f"{input_folder}/{file_name}"

# Save the DataFrame to a CSV file in the specified folder
species_df.to_csv(full_file_path, index=False)

NOW, LETS USE THE QCODES TO RECOVER ALL THE COMPOUNDS (WITH REFERENCES) PRESENT IN EACH QCODE AND SAVE THEM AS INDIVIDUAL CSV FILES FROM LOTUS_DB FROZEN METADATA

In [6]:
# Load the LOTUSDB CSV
lotusdb_df = pd.read_csv(LOTUSDB, low_memory=False)

# Iterate through all CSV files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        input_file = os.path.join(input_folder, filename)

        # Load the CSV file with Q codes
        df_species = pd.read_csv(input_file)

        # Iterate through the Q codes in the species CSV
        for q_code in df_species['wikidata_Qcode_species']:  # Ensure this column name matches your species CSV
            # Skip processing if Q code is 'Not Found'
            if q_code == 'Not Found':
                continue

            # Filter LOTUSDB data for the current Q code
            filtered_lotusdb = lotusdb_df[lotusdb_df['wikidata_Qcode'] == q_code]  # Ensure this column name matches your LOTUSDB CSV

            # Group and aggregate the data
            grouped_df = filtered_lotusdb.groupby("structure_inchikey").agg({
                # Add all the aggregation rules here
                # Example:
                "structure_wikidata": "first",
                "structure_inchi": "first",
                "structure_smiles": "first",
                "structure_molecular_formula": "first",
                "structure_exact_mass": "first",
                "structure_xlogp": "first",
                "structure_smiles_2D": "first",
                "structure_cid": "first",
                "structure_nameIupac": "first",
                "structure_nameTraditional": "first",
                "structure_taxonomy_npclassifier_01pathway": "first",
                "structure_taxonomy_npclassifier_02superclass": "first",
                "structure_taxonomy_npclassifier_03class": "first",
                "organism_wikidata": "first",
                "organism_taxonomy_gbifid": "first",
                "organism_taxonomy_ncbiid": "first",
                "organism_taxonomy_ottid": "first",
                "organism_taxonomy_01domain": "first",
                "organism_taxonomy_02kingdom": "first",
                "organism_taxonomy_03phylum": "first",
                "organism_taxonomy_04class": "first",
                "organism_taxonomy_05order": "first",
                "organism_taxonomy_06family": "first",
                "organism_taxonomy_07tribe": "first",
                "organism_taxonomy_08genus": "first",
                "organism_taxonomy_09species": "first",
                "organism_taxonomy_10varietas": "first",
                "reference_wikidata": lambda x: "|".join(map(str, x)),
                "reference_doi": lambda x: "|".join(map(str, x))
            }).reset_index()
            
            # Create 'chemical_superclass' and 'chemical_class' columns
            grouped_df['chemical_superclass'] = grouped_df['structure_taxonomy_npclassifier_01pathway'] + '-' + grouped_df['structure_taxonomy_npclassifier_02superclass']
            grouped_df['chemical_class'] = grouped_df['structure_taxonomy_npclassifier_01pathway'] + '-' + grouped_df['structure_taxonomy_npclassifier_03class']
            
            # Save the grouped data as a TSV file with the Q code as the filename
            output_subfolder = os.path.join(output_folder, 'species_data')
            #if not os.path.exists(output_subfolder):
            os.makedirs(output_subfolder, exist_ok=True)

            output_filename = os.path.join(output_subfolder, f"{q_code}.tsv")
            grouped_df.to_csv(output_filename, index=False, sep='\t')

            print(f"Saved grouped data for Q code {q_code} to {output_filename}")


Saved grouped data for Q code Q537145 to C:/Users/quirosgu/Desktop/Maytenus/Literature/output_data/species_data\Q537145.tsv
Saved grouped data for Q code Q549692 to C:/Users/quirosgu/Desktop/Maytenus/Literature/output_data/species_data\Q549692.tsv
Saved grouped data for Q code Q1152753 to C:/Users/quirosgu/Desktop/Maytenus/Literature/output_data/species_data\Q1152753.tsv
Saved grouped data for Q code Q1330457 to C:/Users/quirosgu/Desktop/Maytenus/Literature/output_data/species_data\Q1330457.tsv
Saved grouped data for Q code Q1330822 to C:/Users/quirosgu/Desktop/Maytenus/Literature/output_data/species_data\Q1330822.tsv
Saved grouped data for Q code Q2600704 to C:/Users/quirosgu/Desktop/Maytenus/Literature/output_data/species_data\Q2600704.tsv
Saved grouped data for Q code Q2843257 to C:/Users/quirosgu/Desktop/Maytenus/Literature/output_data/species_data\Q2843257.tsv
Saved grouped data for Q code Q2882626 to C:/Users/quirosgu/Desktop/Maytenus/Literature/output_data/species_data\Q2882626.

In [2]:
#Create a unique file containining unique inchi 
import os
import pandas as pd

# Define the directory containing the .tsv files
directory_path = 'C:/Users/quirosgu/Desktop/Maytenus/Literature/output_data/species_data/'  # Adjust this path to your directory containing the .tsv files

# List all .tsv files in the directory
tsv_files = [f for f in os.listdir(directory_path) if f.endswith('.tsv')]

# Initialize an empty list to store DataFrames
dfs = []

# Loop through the list of files and read each one into a DataFrame
for file in tsv_files:
    file_path = os.path.join(directory_path, file)
    df = pd.read_csv(file_path, sep='\t')
    dfs.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dfs, ignore_index=True)

# Drop duplicate values based on the 'structure_inchi' column
unique_df = combined_df.drop_duplicates(subset='structure_inchi')

# Define the output path and save the combined DataFrame to a TSV file
output_path = os.path.join(directory_path, 'Compounds_reported_in_genus.tsv')
unique_df.to_csv(output_path, sep='\t', index=False)

print("Combined and unique DataFrame saved successfully.")


Combined and unique DataFrame saved successfully.


C:\Users\quirosgu\AppData\Local\Temp\ipykernel_27660\3946952289.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(dfs, ignore_index=True)


RECOVER THE FRECUENCE BY CHEMICAL CLASS AND ADDED TO THE GENERAL TABLE

In [7]:
# Step 1: Initialize variables to store data
total_compounds = {}
chemical_classes = {}
chemical_superclasses = {}
references = {}
hyperlinks = {}

# Step 2: Iterate through .tsv files in the folder

species_data_folder = os.path.join(output_folder, 'species_data')
for filename in os.listdir(species_data_folder):
    if filename.endswith(".tsv"):
        qcode = filename.split(".")[0]  # Extract Qcode from the filename

        # Load the .tsv file into a DataFrame
        df_compounds = pd.read_csv(os.path.join(species_data_folder, filename), sep='\t')

        # Step 3: Calculate frequencies of chemical classes for each Qcode (excluding 'not classified')
        filtered_classes = df_compounds[df_compounds['structure_taxonomy_npclassifier_03class'] != 'Not Classified']
        grouped = filtered_classes['structure_taxonomy_npclassifier_03class'].value_counts().reset_index()
        frequencies = grouped.apply(lambda row: f"{row.name} {row['structure_taxonomy_npclassifier_03class']}", axis=1)
        chemical_classes[qcode] = "|".join(frequencies)

        # Step 4: Calculate frequencies of chemical superclasses for each Qcode (excluding 'not classified')
        filtered_sclasses = df_compounds[df_compounds['structure_taxonomy_npclassifier_02superclass'] != 'Not Classified']
        sgrouped = filtered_sclasses['structure_taxonomy_npclassifier_02superclass'].value_counts().reset_index()
        frequencies = sgrouped.apply(lambda row: f"{row.name} {row['structure_taxonomy_npclassifier_02superclass']}", axis=1)
        chemical_superclasses[qcode] = "|".join(frequencies)
        chemical_classes[qcode] = "|".join(frequencies)
        
        # Step 5: Combine references and hyperlinks
        #references[qcode] = "|".join(df_compounds['reference_wikidata'].dropna())
        #hyperlinks[qcode] = "|".join(df_compounds['reference_doi'].dropna())

        # Step 6: Calculate the total number of reported compounds
        #total_compounds[qcode] = len(df_compounds)

# Step 7: Load the general info .csv into a DataFrame
csv_file = None
for filename in os.listdir(input_folder):
    if filename.endswith(".csv"):
        csv_file = os.path.join(input_folder, filename)
        break  # Stop searching after finding the first CSV file

if csv_file is not None:
    # Load the CSV file into the df_general_info DataFrame
    df_general_info = pd.read_csv(csv_file)

# Step 7: Create new columns for combined information
df_general_info['predicted_class'] = df_general_info['wikidata_Qcode_species'].map(chemical_classes)
df_general_info['predicted_superclass'] = df_general_info['wikidata_Qcode_species'].map(chemical_superclasses)
#df_general_info['references'] = df_general_info['wikidata_Qcode_species'].map(references)
#df_general_info['referenceLabel'] = df_general_info['wikidata_Qcode_species'].map(hyperlinks)

# Step 7: Save the updated DataFrame to the .csv file

# Replace values matching the pattern with an empty cell
df_general_info['predicted_class'] = df_general_info['predicted_class'].replace('index|predicted_class', '')
df_general_info['predicted_superclass'] = df_general_info['predicted_superclass'].replace('index|predicted_superclass', '')

#add reported compounds to each Q code
LotusDB_rc = pd.read_csv(LOTUSDB_rc)

spDB = LotusDB_rc[['wikidata_Qcode', 'Reported_comp_Species']].drop_duplicates()
gDB = LotusDB_rc[['organism_taxonomy_08genus', 'Reported_comp_Genus']].drop_duplicates()
fDB = LotusDB_rc[['organism_taxonomy_06family', 'Reported_comp_Family']].drop_duplicates()

df = pd.merge(df_general_info, spDB,
            how= 'left', left_on='wikidata_Qcode_species', right_on='wikidata_Qcode')#.drop_duplicates(subset=['CODE'])
df.drop('wikidata_Qcode', axis=1, inplace=True)

#df = pd.merge(df, gDB,
#            how= 'left', left_on='genus', right_on='organism_taxonomy_08genus')#.drop_duplicates(subset=['full_species'])
#df.drop('organism_taxonomy_08genus', axis=1, inplace=True)

#df = pd.merge(df, fDB,
#            how= 'left', left_on='family', right_on='organism_taxonomy_06family')#.drop_duplicates(subset=['full_species'])
#df.drop('organism_taxonomy_06family', axis=1, inplace=True)

output_csv_file = output_folder + 'Full_results.csv'
df.to_csv(output_csv_file, index=False, sep =',')

KeyError: "['wikidata_Qcode'] not in index"

# BARPLOT CHEMICAL_SUPERCLASS - SPECIES 

In [8]:
import plotly.express as px

def create_stacked_barplot(agg_data, unique_species, unique_superclasses, color_map, output_folder):
    """
    Create a stacked barplot with custom color palette.

    Parameters:
    agg_data (DataFrame): Dataframe containing aggregated data.
    unique_species (list): List of unique species.
    unique_superclasses (list): List of unique superclasses.
    color_map (dict): Custom color map for the barplot.
    output_folder (str): Path to the output folder.

    Returns:
    None
    """
    # Create the stacked barplot with the custom color palette
    fig = px.bar(agg_data, y='species', x='recurrence',
                 title='Stacked Barplot of Predicted Superclasses Occurrence for Species',
                 labels={'recurrence': 'Recurrence'},
                 color='chemical_superclass',
                 color_discrete_map=color_map,
                 category_orders={'species': unique_species, 'chemical_superclass': unique_superclasses},
                 orientation='h')

    # Modify the y-axis label
    fig.update_yaxes(title_text='<i>Species</i>')

    # Set species labels in italics
    fig.update_layout(yaxis=dict(tickmode='array',
                                  tickvals=list(range(len(unique_species))),
                                  ticktext=[f'<i>{species}</i>' for species in unique_species]
                                  ))

    # Set a white background
    fig.update_layout(plot_bgcolor='white')

    # Modify the size of the figure
    fig.update_layout(width=1500, height=1500)

    # Save the figure as an HTML file
    fig.write_html(f'{output_folder}Wikidata_superclass_barplot_species.html')

    # Show the figure
    fig.show()

# Example usage:
# create_stacked_barplot(agg_data, unique_species, unique_superclasses, color_map, output_folder)


In [9]:
# Step 1: Read data from all .tsv files in the output_folder
all_data = pd.concat([pd.read_csv(os.path.join(output_folder, filename), sep='\t') for filename in os.listdir(os.path.join(output_folder, 'species_data')) if filename.endswith(".tsv")])

# Step 2: Rename the "organism_taxonomy_09species" column to "species"
all_data.rename(columns={'organism_taxonomy_09species': 'species'}, inplace=True)

# Remove 'API Error-API Error' and 'Not Classified-Not Classified'
all_data = all_data[~all_data['chemical_superclass'].isin(['API Error-API Error', 'Not Classified-Not Classified'])]

# Apply the function to each row
all_data[['Pathway', 'superclass']] = all_data.apply(lambda row: split_chemical_superclass(row), axis=1, result_type='expand')

# Step 4: Process data for color mapping
color_map = {}
for pathway, superclasses in all_data.groupby('Pathway')['superclass'].unique().items():
    shades = generate_shades(pathway, len(superclasses))
    for superclass, shade in zip(superclasses, shades):
        color_map[f"{pathway}-{superclass}"] = shade

# Step 5: Group and aggregate data to calculate recurrence
agg_data = all_data.groupby(['species', 'chemical_superclass']).size().reset_index(name='recurrence')

# Convert 'species' column to categorical data
agg_data['species'] = pd.Categorical(agg_data['species'], categories=agg_data['species'].unique(), ordered=True)

# Get unique species names
unique_species = agg_data['species'].unique()

# Get unique chemical superclasses and sort them alphabetically
unique_superclasses = sorted(agg_data['chemical_superclass'].unique())

# Calculate total recurrence for each species
total_recurrence_per_species = agg_data.groupby('species')['recurrence'].sum()

create_stacked_barplot(agg_data, unique_species, unique_superclasses, color_map, output_folder)


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/quirosgu/Desktop/Maytenus/Literature/output_data/Q10327859.tsv'

In [10]:
# Step 1: Read data from all .tsv files in the output_folder
all_data = pd.concat([pd.read_csv(os.path.join(output_folder, 'species_data', filename), sep='\t') for filename in os.listdir(os.path.join(output_folder, 'species_data')) if filename.endswith(".tsv")])

# Step 2: Rename the "organism_taxonomy_09species" column to "species"
all_data.rename(columns={'organism_taxonomy_09species': 'species'}, inplace=True)

# Remove 'API Error-API Error' and 'Not Classified-Not Classified'
all_data = all_data[~all_data['chemical_superclass'].isin(['API Error-API Error', 'Not Classified-Not Classified'])]

# Apply the function to each row
all_data[['Pathway', 'superclass']] = all_data.apply(lambda row: split_chemical_superclass(row), axis=1, result_type='expand')

# Step 4: Process data for color mapping
color_map = {}
for pathway, superclasses in all_data.groupby('Pathway')['superclass'].unique().items():
    shades = generate_shades(pathway, len(superclasses))
    for superclass, shade in zip(superclasses, shades):
        color_map[f"{pathway}-{superclass}"] = shade

# Step 5: Group and aggregate data to calculate recurrence
agg_data = all_data.groupby(['species', 'chemical_superclass']).size().reset_index(name='recurrence')

# Normalize the recurrence values within each species group
agg_data['recurrence_normalized'] = agg_data.groupby('species')['recurrence'].transform(lambda x: x / x.sum())*100

# Convert 'species' column to categorical data
agg_data['species'] = pd.Categorical(agg_data['species'], categories=agg_data['species'].unique(), ordered=True)

# Get unique species names
unique_species = agg_data['species'].unique()

# Get unique chemical superclasses and sort them alphabetically
unique_superclasses = sorted(agg_data['chemical_superclass'].unique())

# Calculate total recurrence for each species
total_recurrence_per_species = agg_data.groupby('species')['recurrence'].sum()

# Create the stacked barplot with the custom color palette
fig = px.bar(agg_data, y='species', x='recurrence_normalized',
             title='Normalized Stacked Barplot of Predicted Superclasses Occurrence for Species',
             labels={'recurrence_normalized': 'Recurrence'},
             color='chemical_superclass',
             color_discrete_map=color_map,
             category_orders={'species': unique_species, 'chemical_superclass': unique_superclasses},
             orientation='h')

# Modify the y-axis label
fig.update_yaxes(title_text='<i>Species<i>')

# Set species labels in italics
fig.update_layout(yaxis=dict(tickmode='array',
                              tickvals=list(range(len(unique_species))),
                              ticktext=[f'<i>{species}</i>' for species in unique_species]
                              ))

# Set a white background
fig.update_layout(plot_bgcolor='white')

# Get the maximum value of the x-axis
max_x = agg_data['recurrence_normalized'].max()

for species, total_recurrence in total_recurrence_per_species.items():
    # Find the maximum recurrence for this species
    max_recurrence = agg_data[agg_data['species'] == species]['recurrence_normalized'].max()
    
    # Position the annotation at the end of the bar with a slight offset
    x_position = max_x + 2
    if x_position > 100:  # Ensure the label is not outside the plot area
        x_position = 100  # Set it to 100 if it exceeds the maximum x-axis value

    # Add annotation to the plot
    fig.add_annotation(
        x=x_position,
        y=species,
        text=f'Total compounds: {total_recurrence}',
        showarrow=False,
        font=dict(size=10, color='black'),
        xanchor='left',
        yanchor='middle'
    )

# Modify the size of the figure
fig.update_layout(width=1500, height=1500)

# Save the figure as an HTML file
fig.write_html(f'{output_folder}Wikidata_superclass_barplot_species_normalized.html')

# Show the figure
fig.show()


C:\Users\quirosgu\AppData\Local\Temp\ipykernel_32328\1941301309.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_data = pd.concat([pd.read_csv(os.path.join(output_folder, 'species_data', filename), sep='\t') for filename in os.listdir(os.path.join(output_folder, 'species_data')) if filename.endswith(".tsv")])
C:\Users\quirosgu\AppData\Local\Temp\ipykernel_32328\1941301309.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  total_recurrence_per_species = agg_data.groupby('species')['recurrence'].sum()


# BARPLOT PATHWAY - SPECIES 

In [11]:

# Step 1: Initialize an empty DataFrame to store all data
all_data = pd.DataFrame(columns=['organism_taxonomy_09species', 'structure_taxonomy_npclassifier_01pathway'])

# Step 2: Read data from all .tsv files in the output_folder
species_data_folder = os.path.join(output_folder, 'species_data')
for filename in os.listdir(species_data_folder):
    if filename.endswith(".tsv"):
        file_data = pd.read_csv(os.path.join(species_data_folder, filename), sep='\t')
        all_data = pd.concat([all_data, file_data])

# Step 3: Rename the "queried_taxaLabel" column to "species"
all_data.rename(columns={'organism_taxonomy_09species': 'species'}, inplace=True)
all_data.rename(columns={'structure_taxonomy_npclassifier_01pathway': 'Pathway'}, inplace=True)

# Remove 'API Error-API Error' and 'Not Classified-Not Classified'
all_data = all_data[(all_data['Pathway'] != 'API Error') & (all_data['Pathway'] != 'Not Classified')]

# Step 4: Group and aggregate data to calculate recurrence
agg_data = all_data.groupby(['species', 'Pathway']).size().reset_index(name='recurrence')

# Convert 'species' column to categorical data
agg_data['species'] = pd.Categorical(agg_data['species'], categories=agg_data['species'].unique(), ordered=True)

# Get unique species names
unique_species = agg_data['species'].unique()

# Get unique pathways and sort them alphabetically
unique_pathways = sorted(agg_data['Pathway'].unique())

# Calculate total recurrence for each species
total_recurrence_per_species = agg_data.groupby('species')['recurrence'].sum()


# Step 5: Create the stacked barplot with custom colors
fig = px.bar(agg_data, y='species', x='recurrence',
             title='Stacked Barplot of Predicted Pathways Ocurrence for Species',
             labels={'recurrence': 'Recurrence'},
             color='Pathway',
             color_discrete_map=pathway_colors,  # Use custom colors
             category_orders={'species': unique_species, 'Pathway' :unique_pathways },
             orientation='h'
            )

# Modify the y-axis label
fig.update_yaxes(title_text='<i>Species<i>')

# Set species labels in italics
fig.update_layout(yaxis=dict(tickmode='array', 
                              tickvals=list(range(len(unique_species))), 
                              ticktext=[f'<i>{species}</i>' for species in unique_species]
                              ))

# Set a white background
fig.update_layout(plot_bgcolor='white')

# Modify the size of the figure
fig.update_layout(width=1500, height=1500)

# Save the figure as an HTML file
fig.write_html(f'{output_folder}Wikidata_pathway_barplot_species.html')

# Show or save the figure
fig.show()


C:\Users\quirosgu\AppData\Local\Temp\ipykernel_32328\4072444705.py:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

C:\Users\quirosgu\AppData\Local\Temp\ipykernel_32328\4072444705.py:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

C:\Users\quirosgu\AppData\Local\Temp\ipykernel_32328\4072444705.py:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the resu

In [12]:

#create a normalized figure

# Normalize the recurrence values within each species group
agg_data['recurrence_normalized'] = agg_data.groupby('species')['recurrence'].transform(lambda x: x / x.sum())*100

# Convert 'species' column to categorical data
agg_data['species'] = pd.Categorical(agg_data['species'], categories=agg_data['species'].unique(), ordered=True)

# Get unique species names
unique_species = agg_data['species'].unique()

# Get unique pathways and sort them alphabetically
unique_pathways = sorted(agg_data['Pathway'].unique())

# Calculate total recurrence for each species
total_recurrence_per_species = agg_data.groupby('species')['recurrence'].sum()


# Step 5: Create the stacked barplot with custom colors
fig = px.bar(agg_data, y='species', x='recurrence_normalized',
             title='Normalized Stacked Barplot of Predicted Pathways Ocurrence for Species',
             labels={'recurrence_normalized': 'Recurrence'},
             color='Pathway',
             color_discrete_map=pathway_colors,  # Use custom colors
             category_orders={'species': unique_species, 'Pathway' :unique_pathways },
             orientation='h'
            )

# Modify the y-axis label
fig.update_yaxes(title_text='<i>Species<i>')

# Set species labels in italics
fig.update_layout(yaxis=dict(tickmode='array', 
                              tickvals=list(range(len(unique_species))), 
                              ticktext=[f'<i>{species}</i>' for species in unique_species]
                              ))
# Get the maximum value of the x-axis
max_x = agg_data['recurrence_normalized'].max()

for species, total_recurrence in total_recurrence_per_species.items():
    # Find the maximum recurrence for this species
    max_recurrence = agg_data[agg_data['species'] == species]['recurrence_normalized'].max()
    
    # Position the annotation at the end of the bar with a slight offset
    x_position = max_x + 2
    if x_position > 100:  # Ensure the label is not outside the plot area
        x_position = 100  # Set it to 100 if it exceeds the maximum x-axis value

    # Add annotation to the plot
    fig.add_annotation(
        x=x_position,
        y=species,
        text=f'Total compounds: {total_recurrence}',
        showarrow=False,
        font=dict(size=10, color='black'),
        xanchor='left',
        yanchor='middle'
    )
    
# Set a white background
fig.update_layout(plot_bgcolor='white')

# Modify the size of the figure
fig.update_layout(width=1500, height=1500)

# Save the figure as an HTML file
fig.write_html(f'{output_folder}Wikidata_pathway_barplot_normalized.html')

# Show or save the figure
fig.show()


C:\Users\quirosgu\AppData\Local\Temp\ipykernel_32328\848878991.py:4: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\quirosgu\AppData\Local\Temp\ipykernel_32328\848878991.py:16: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [13]:
import pandas as pd
import plotly.express as px

# Create a DataFrame with all possible combinations of species and pathways
all_species = all_data['species'].unique()
all_pathways = all_data['Pathway'].unique()
all_combinations = pd.DataFrame([(species, pathway) for species in all_species for pathway in all_pathways],
                                columns=['species', 'Pathway'])

# Merge the all_combinations DataFrame with the aggregated data to include all species
merged_data = pd.merge(all_combinations, agg_data, on=['species', 'Pathway'], how='left')

# Fill missing values (NaN) with 0 for recurrence and recurrence_normalized columns
merged_data['recurrence'].fillna(0, inplace=True)
merged_data['recurrence_normalized'].fillna(0, inplace=True)

# Pivot the merged data to have 'species' as rows and 'Pathway' as columns
pivot_data = merged_data.pivot_table(index='species', columns='Pathway', values='recurrence_normalized', fill_value=0)

# Create the heatmap
fig = px.imshow(pivot_data,
                labels=dict(x="Pathway", y="Species", color="Normalized Recurrence"),
                x=pivot_data.columns,
                y=pivot_data.index,
                color_continuous_scale='tealrose',  # You can choose any other color scale
                title='Heatmap of Normalized Recurrence of Pathways for Species')

# Modify the size of the figure
fig.update_layout(width=1500, height=1500)

# Save the figure as an HTML file
fig.write_html(f'{output_folder}Wikidata_pathway_heatmap_species_normalized.html')

# Show the figure
fig.show()


C:\Users\quirosgu\AppData\Local\Temp\ipykernel_32328\876206520.py:14: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\quirosgu\AppData\Local\Temp\ipykernel_32328\876206520.py:15: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, in

In [14]:
# Convert 'species' and 'chemical_superclass' to categorical data for proper ordering
agg_data['species'] = pd.Categorical(agg_data['species'], categories=sorted(agg_data['species'].unique()))
agg_data['chemical_superclass'] = pd.Categorical(agg_data['chemical_superclass'], categories=sorted(agg_data['chemical_superclass'].unique()))

# Pivot the data to have 'species' as rows and 'chemical_superclass' as columns
pivot_data = agg_data.pivot_table(index='species', columns='chemical_superclass', values='recurrence_normalized', fill_value=0)
# Multiply the normalized values by 100 to scale them to 100
pivot_data *= 100

# Create the heatmap
fig = px.imshow(pivot_data,
                labels=dict(x="Chemical Superclass", y="Species", color="Normalized Recurrence"),
                x=pivot_data.columns,
                y=pivot_data.index,
                color_continuous_scale='tealrose',  # You can choose any other color scale
                title='Heatmap of Normalized Recurrence of Chemical Superclasses for Species')

# Modify the size of the figure
fig.update_layout(width=1500, height=1500)

# Save the figure as an HTML file
fig.write_html(f'{output_folder}Wikidata_superclass_heatmap_species_normalized.html')

# Show the figure
fig.show()


KeyError: 'chemical_superclass'